In [1]:
from PYME.LMVis import VisGUI

%gui wx

ERROR:PYME.recipes.modules:Error loading plugin: cc_drift_cor.plugins.recipes.processing
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\recipes\modules.py", line 42, in <module>
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'cc_drift_cor'
ERROR:PYME.recipes.modules:Error loading plugin: cc_drift_cor.plugins.recipes.localisations
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\recipes\modules.py", line 42, in <module>
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'cc_drift_cor'
ERROR:PYME.recipes.modules:Error loading plugin: pyme_4pi.recipe_modules.conversion
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\recipes\modules.py", line 42, in <module>
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'pyme_4pi'
ERROR:PYME.recipes.modules:Error loading plugi

Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.localisations
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Trying to load 3rd party recipe module pyme_4pi.recipe_modules.conversion
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.chung_cc
Trying to load 3rd party recipe module oidic.recipe_modules.reconstruct
Loaded 3rd party recipe module oidic.recipe_modules.reconstruct
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.io


INFO:OpenGL.GLUT.special:Using NT-specific GLUT calls with exit callbacks


quaternion module not found, disabling custom clip plane orientations
quaternion module not found, disabling custom clip plane orientations


In [2]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

import numpy as np
import os

DEBUG:PYME.LMVis.visCore:Creating VisGUI menu bar
DEBUG:PYME.LMVis.Extras:Initializing QPobjectSegment plugin
DEBUG:PYME.LMVis.Extras:Initializing animation plugin
DEBUG:PYME.LMVis.Extras:Initializing clusterAnalysis plugin
DEBUG:PYME.LMVis.Extras:Initializing dockedPanel plugin
DEBUG:PYME.LMVis.Extras:Initializing extra_layers plugin
DEBUG:PYME.LMVis.Extras:Initializing fiducial plugin
DEBUG:PYME.LMVis.Extras:Initializing flagZMotionArtifacts plugin
DEBUG:root:Adding menu items for event filters
DEBUG:PYME.LMVis.Extras:Initializing fusionUtils plugin
DEBUG:PYME.LMVis.Extras:Initializing multiColourAnalysis plugin
DEBUG:PYME.LMVis.Extras:Initializing multiviewMapping plugin
DEBUG:root:Adding menu items for multi-view manipulation
DEBUG:PYME.LMVis.Extras:Initializing objectMeasurements plugin
DEBUG:PYME.LMVis.Extras:Initializing particleTracking plugin
DEBUG:PYME.LMVis.Extras:Initializing perFrameVariable plugin
DEBUG:PYME.LMVis.Extras:Initializing photophysics plugin
DEBUG:PYME.LMVis.E

New Canvas
Saved Views are deprecated -  use the ones with animation instead and remove me!


ERROR:PYME.LMVis.Extras:Error loading plugin: pyme_4pi.visgui_modules.precision
Traceback (most recent call last):
  File "c:\users\zrc4\code\python-microscopy\PYME\LMVis\Extras\__init__.py", line 46, in InitPlugins
    m = __import__(mn, fromlist=mn.split('.')[:-1])
ModuleNotFoundError: No module named 'pyme_4pi'


Creating fold panel
GenPanels
Creating datasource panel
RecipeView._layout
Input datasources:  []
Image list size: 1
Creating layers GUI
height:  0
NB best size: wx.Size(200, 0)
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  []
recipe.execute()
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  []
0
OpenGL - Version: b'4.6.0 NVIDIA 442.50'
Shader - Version: b'4.60 NVIDIA'
GL_MAX_SAMPLES: 32, GL_SAMPLES: 4


DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.DefaultShaderProgram.DefaultShaderProgram'>


Ev Idle
('points', 't')
refreshed


In [3]:
"""
This compares ICTM mesh quality as a function of parameters.
"""

# Where to save the intermediate files generated 
save_fp = 'C:\\Users\\zrc4\\Downloads\\test_ictm_7'
if not os.path.exists(save_fp):
    os.mkdir(save_fp)

# three way junction generation parameters
centroid = np.array([0,0,0])
h, r = 500, 50       # capsule length, capsule radius
smoothing = r      # ~h/2 creates a three-way junction with a "sheet" (but it's puffy like seaweed)
# loc precision
psf_width = 250.0
mean_photon_count = 300.0

# octree-based reconstruction parameters
cull_inner_surfaces = True
n_points_min =30
remesh = True
repair = False
smooth_curvature = True
density = 6e-6

# screened poisson reconstruction parameters
fulldepth=5


In [4]:
"""
Generate three-way junction
"""

from ch_shrinkwrap.shape import ThreeWayJunction

twj = ThreeWayJunction(h, r, centroid, smoothing)

In [5]:
"""
Generate and save three-way junction ponts
"""
import time

from ch_shrinkwrap import util

cap_points = twj.points(p=0.001, psf_width=psf_width, mean_photon_count=mean_photon_count, resample=True)
cap_sigma = twj._noise

no = 0.1
scale = 1.2
bbox = [np.min(cap_points[:,0]), np.min(cap_points[:,1]), 
        np.min(cap_points[:,2]), np.max(cap_points[:,0]),
        np.max(cap_points[:,1]), np.max(cap_points[:,2])]
bbox = [scale*x for x in bbox]
xl, yl, zl, xu, yu, zu = bbox
xn, yn, zn = xu-xl, yu-yl, zu-zl
ln = int(no*len(cap_points)/(1.0-no))
noise_points = np.random.rand(ln,3)*(np.array([xn,yn,zn])[None,:]) + (np.array([xl,yl,zl])[None,:])
noise_sigma = util.noise(noise_points.shape, model='poisson', psf_width=psf_width, mean_photon_count=mean_photon_count)

points = np.vstack([cap_points,noise_points])
sigma = np.vstack([cap_sigma,noise_sigma])
s = np.sqrt((sigma*sigma).sum(1))

points_time = time.strftime('%Y%d%m_%HH%M')
points_fn = f"twj_h{h}_r{r}_smoothing{smoothing}_{points_time}".replace('.','_')+".txt"
points_fp = save_fp + '/' + points_fn
np.savetxt(points_fp, np.vstack([points.T,s]).T, header="x y z sigma")

In [6]:
# Now we need PYMEVis
pymevis.OpenFile(points_fp)

# Octree renderer (original mesh based on parameters above)
from PYME.LMVis.Extras.extra_layers import gen_octree_from_points
from PYME.recipes.surface_fitting import DualMarchingCubes

oc_name = gen_octree_from_points(pymevis)
surf_name, surf_count = pipeline.new_ds_name('surf', return_count=True)

recipe = pipeline.recipe
dmc = DualMarchingCubes(recipe, invalidate_parent=False, input=oc_name, output=surf_name,
                       threshold_density=density, n_points_min=n_points_min, remesh=remesh,
                       repair=repair, cull_inner_surfaces=cull_inner_surfaces, smooth_curvature=smooth_curvature)
recipe.add_modules_and_execute([dmc,])

octree_fp = points_fp.split('.txt')[0] + '_octree.stl'
pipeline.dataSources[dmc.output].to_stl(octree_fp)

Creating Pipeline
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  ['FitResults']
RecipeView._layout


SystemError: <class 'wx._core.WindowDestroyEvent'> returned a result with an error set

RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

...

Input datasources:  ['FitResults', 'octree0']


DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.DefaultShaderProgram.OpaquePointShaderProgram'>


lw update
Creating layers GUI
hl update
height:  320
NB best size: wx.Size(222, 320)


DEBUG:PYME.LMVis.visCore:Added layer, datasouce=output


lw update
hl update
hl update


In [ ]:
"""
At this point, we will start from the original surface and iterate using
different parameters.
"""
# Shrinkwrap renderer
from ch_shrinkwrap import _membrane_mesh as membrane_mesh

max_iters = np.hstack([5, np.logspace(1,2,10).astype(int)])
step_size = np.logspace(0,2,10)
search_k = np.hstack([2,np.arange(0,105,5)[1:]])[:5]
remesh_every = np.array([5,10,20,50,100])

# max_iters = np.array([10])
# remesh_every = np.array([5])
# step_size = np.array([12.9])
# search_k = np.array([20])

for it in max_iters:
    for lam in step_size:
        for k in search_k:
            for re in remesh_every:
                # Copy the mesh over
                mesh = membrane_mesh.MembraneMesh(mesh=pipeline.dataSources[dmc.output])

                # set params
                mesh.max_iter = it
                mesh.step_size = lam
                mesh.search_k = k
                mesh.remesh_frequency = re

                start = time.time()
                mesh.shrink_wrap(points, s, method='ictm')
                stop = time.time()
                duration = stop-start

                wrap_fp = points_fp.split('.txt')[0] + "_".join(f"_iters{it}_remesh{re}_lambda{lam:.1f}_searchk{k}_noise{no}_ntriangles{mesh.faces.shape[0]}_duration{duration:.1f}_ictm".split('.')) + ".stl"
                mesh.to_stl(wrap_fp)

Test Statistic 0.807285
Test Statistic 0.976583
Test Statistic 0.532299
Test Statistic 0.261967
Test Statistic 0.095714
Split count: 70
Flip count: 66 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 127[1 failed]
Flip count: 27 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Split count: 49
Flip count: 47 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 53[0 failed]
Flip count: 24 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Split count: 31
Flip count: 26 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 43[0 failed]
Flip count: 14 [0 failed]
Flip count: 1 [0 failed]
Flip count: 0 [0 failed]
Split count: 56
Flip count: 44 [0 failed]
Flip count: 2 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 47[0 failed]
Flip count: 20 [0 failed]
Flip count: 0 [0 failed]
Split count: 34
Flip count: 27 [0 failed]
Flip count: 3 [0 failed]
Flip count: 0 [0 failed]
Collapse count: 41[0 failed]
Flip count: 13 [0 failed]
Flip count: 1 [0 fa

In [ ]:
"""
Now calculate SPR.
"""

def screened_poisson(points_fp, rowstoskip=1, strformat='X Y Z Reflectance', separator='SPACE',
                     colorformat='[0-255]', onerror='skip', k=10, smoothiter=0,
                     flipflag=False, viewpos=[0,0,0], depth=8, fulldepth=5,
                     cgdepth=0, scale=1.1, samplespernode=1.5, pointweight=4, 
                     iters=8, confidence=False, preclean=False):
    """
    Run screened poisson reconstruction on a set of points, using meshlab.
    
    For more information on these parameters, see meshlab.
    
    Parameters
    ----------
    points_fp : str
        Path to text file containing a point cloud represented as a set of XYZ coordinates
    see meshlab
    
    Returns
    -------
    str
        File path to STL of reconstruction
        
    """
    import pymeshlab as ml

    ms = ml.MeshSet()  # create a mesh
    ms.load_new_mesh(file_name=points_fp,
                     rowtoskip=rowstoskip,
                     strformat=strformat,
                     separator=separator,
                     rgbmode=colorformat,
                     onerror=onerror)  # load points
    start = time.time()
    # compute normals
    ms.compute_normals_for_point_sets(k=k,  # number of neighbors
                                      smoothiter=smoothiter,
                                      flipflag=flipflag,
                                      viewpos=viewpos)
    # run SPR
    ms.surface_reconstruction_screened_poisson(visiblelayer=False,
                                               depth=depth,
                                               fulldepth=fulldepth,
                                               cgdepth=cgdepth,
                                               scale=scale,
                                               samplespernode=samplespernode,
                                               pointweight=pointweight,
                                               iters=iters,
                                               confidence=confidence,
                                               preclean=preclean)
    stop = time.time()
    duration = stop-start
    # save surface
    surface_fp = points_fp.split('.txt')[0] + "_".join(f"_searchk{k}_depth{depth}_scale{scale}_samplespernode{samplespernode}_pointweight{pointweight}_iters{iters}_noise{no}_duration{duration:.1f}_spr".split('.')) + ".stl"
    ms.save_current_mesh(file_name=surface_fp, unify_vertices=True)
    
    return surface_fp

max_iters = np.arange(4,15,2)
depth = np.arange(4,15,2,dtype=int)
search_k = np.hstack([2,np.arange(0,105,5)[1:]])[:7]
scale=np.arange(5,16,2)/10
samplespernode=np.arange(1,20,0.5)
pointweight=np.arange(0,10)

# max_iters = np.array([8],dtype=int)
# depth = np.array([8],dtype=int)
# search_k = np.array([10])
# scale=np.array([1.1])
# samplespernode=np.array([1.5])
# pointweight=np.array([4])

for it in max_iters:
    for k in search_k:
        for d in depth:
            for sc in scale:
                for spn in samplespernode:
                    for wt in pointweight:
                        spr_fp = screened_poisson(points_fp, k=k, depth=d, fulldepth=fulldepth, 
                                                  scale=sc, samplespernode=spn, pointweight=wt, iters=it)